In [3]:
pip install transformers torch accelerate bitsandbytes sentencepiece

In [4]:
pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.7-cp311-cp311-linux_x86_64.whl size=4552823 sha256=700a3385eb5dbd11a40e6323593aed87f7f291220502db49b2c82b00d089243e
  Stored in directory: /root/.cache/pip/wheels/eb/82/79/ac77fcd49324b75ae6aa18e63a87cf9da4371a57e2cdc8dc03
Successfully built llama-cpp-python


# **Move Model in a Colab Space**

In [5]:
import shutil

# ✅ Model ko Google Drive se Local Colab Storage Pe Copy Karo
model_source = "/content/drive/MyDrive/Llama Model/llama-2-7b.Q4_K_M.gguf"
model_dest = "/content/llama-2-7b.Q4_K_M.gguf"

shutil.copy(model_source, model_dest)
print("✅ Model copied to local storage!")

# ✅ Ab local path se load karo
model_path = "/content/llama-2-7b.Q4_K_M.gguf"
llm = Llama(model_path=model_path)

✅ Model copied to local storage!


NameError: name 'Llama' is not defined

# **Embedding a Data**

In [7]:
import pandas as pd
from sentence_transformers import SentenceTransformer

# Initialize model
embedder = SentenceTransformer("all-mpnet-base-v2")

# Load data
df = pd.read_csv("product-data.csv")

# Create combined features
df["features"] = "Color: " + df["color"] + ", Memory: " + df["memory"]

# Create description with PKR formatting
df["description"] = (
    df["product_title"] + " | " +
    "Brand: " + df["brand"] + " | " +
    "Price: PKR " + df["price"].astype(str) + " | " +
    "Features: " + df["features"]
)

# Generate embeddings
df["embedding"] = df["description"].apply(lambda x: embedder.encode(x).tolist())

# Save with required columns
df[['product_title', 'brand', 'price', 'features', 'embedding']].to_pickle("product_embeddings.pkl")
print("✅ Embeddings generated with Pakistan-specific formatting!")

✅ Embeddings generated with Pakistan-specific formatting!


# **Model Creation**

In [8]:
\import os
import pandas as pd
import pickle
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from llama_cpp import Llama

# ----- Configuration & File Paths -----
CSV_FILE = "product-data.csv" # CSV file containing product data
EMBEDDING_FILE = "product_embeddings.pkl" # File to cache computed embeddings
DEBUG_MODE = False # Set True for debugging

# Load a robust embedding model
embedding_model_name = "all-mpnet-base-v2"
embedder = SentenceTransformer(embedding_model_name)

# Path to your LLaMA model (update this path accordingly)
llama_model_path = "/content/llama-2-7b.Q4_K_M.gguf"
llama = Llama(model_path=llama_model_path, n_ctx=4096)

# ----- Load Product Data and Embeddings -----
if os.path.exists(EMBEDDING_FILE):
    with open(EMBEDDING_FILE, "rb") as f:
        df = pickle.load(f)
    if DEBUG_MODE:
        print("✅ Loaded precomputed embeddings from pickle.")
else:
    df = pd.read_csv(CSV_FILE)
    # Create a description column if not already present
    df["description"] = (
        df["product_title"] + " - " + df["brand"] + " - " +
        df["color"] + " - " + df["memory"] + " - Rs." + df["price"].astype(str)
    )
    df["embedding"] = df["description"].apply(lambda text: embedder.encode(text).tolist())
    with open(EMBEDDING_FILE, "wb") as f:
        pickle.dump(df, f)
    if DEBUG_MODE:
        print("✅ Embeddings computed and saved.")

# ----- Product Recommendation Function -----
def recommend_product(query: str, top_k: int = 1) -> str:
    """
    Uses precomputed embeddings and the LLaMA model to provide a single, concise, final product
    recommendation in a friendly, ChatGPT-like tone based solely on CSV product data.
    """
    # Compute the embedding for the user query
    query_emb = embedder.encode(query)

    # Calculate cosine similarity for each product and select the best match.
    df["similarity"] = df["embedding"].apply(lambda emb: cosine_similarity([emb], [query_emb])[0][0])
    df_sorted = df.sort_values("similarity", ascending=False)
    top_products = df_sorted.head(top_k)

    # Build product details string for the best match.
    if top_products.iloc[0]["similarity"] < 0.2:
        product_details = "No matching products found in our catalog."
    else:
        # Since top_k is 1, loop should run only once.
        product_details = ""
        for idx, row in top_products.iterrows():
            product_details = (
                f"Product Name: {row['product_title']}\n"
                f"Brand: {row['brand']}\n"
                f"Color: {row.get('color', 'N/A')}\n"
                f"Memory: {row.get('memory', 'N/A')}\n"
                f"Price: {row.get('price', 'N/A')}\n"
            )

    # Construct a prompt with very explicit instructions.
    prompt = f"""
You are a friendly and knowledgeable product recommendation assistant.
Based ONLY on the product data provided below (from a CSV file), answer the user's query in a natural, conversational, and precise manner.
Do NOT include any of the context, headers, or product data in your output.if user ask for a multiple products
Provide ONLY one final answer exactly in the format specified below.use a conversational tone and answer the uqeries accordingly which are related to products.

Final Answer Format:
Product Name: [Name]
Brand: [Brand]
Color: [Color]
Memory: [Memory]
Price: [Price]

User Query: "{query}"
Product Data:
{product_details}

Now, provide ONLY your final answer below (do not repeat any above text):
"""

    if DEBUG_MODE:
        print("DEBUG - Prompt for LLaMA:")
        print(prompt)

    # Generate the recommendation using LLaMA.
    response = llama(
        prompt,
        max_tokens=500,      # Increase max_tokens if needed
        temperature=0.4,     # Lower temperature for a more deterministic response
        stop=["\n\n"]       # Use a stop token to cut off extra output
    )

    if DEBUG_MODE:
        print("DEBUG - Raw LLaMA Response:")
        print(response)

    # Extract and return the generated text.
    if "choices" in response and len(response["choices"]) > 0:
        recommendation = response["choices"][0]["text"].strip()
    else:
        recommendation = "❌ No recommendation generated."

    return recommendation

# ----- Main Loop -----
if __name__ == "__main__":
    print("Product Recommendation Bot (type 'exit' to quit)")
    while True:
        user_query = input("\n🔍 Enter your product query: ")
        if user_query.lower() == "exit":
            print("👋 Exiting...")
            break
        rec = recommend_product(user_query)
        print("\n🤖 AI Recommendation:\n")
        print(rec)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /content/llama-2-7b.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32            

Product Recommendation Bot (type 'exit' to quit)

🔍 Enter your product query: Samsung Galaxy A06


llama_perf_context_print:        load time =   89179.42 ms
llama_perf_context_print: prompt eval time =   89178.60 ms /   223 tokens (  399.90 ms per token,     2.50 tokens per second)
llama_perf_context_print:        eval time =   27097.98 ms /    39 runs   (  694.82 ms per token,     1.44 tokens per second)
llama_perf_context_print:       total time =  116299.51 ms /   262 tokens



🤖 AI Recommendation:

Product Name: Samsung Galaxy A06
Brand: Samsung
Color: N/A
Memory: N/A
Price: 31500

🔍 Enter your product query: Samsung Galaxy A05s


Llama.generate: 158 prefix-match hit, remaining 67 prompt tokens to eval
llama_perf_context_print:        load time =   89179.42 ms
llama_perf_context_print: prompt eval time =   26478.62 ms /    67 tokens (  395.20 ms per token,     2.53 tokens per second)
llama_perf_context_print:        eval time =   31455.34 ms /    44 runs   (  714.89 ms per token,     1.40 tokens per second)
llama_perf_context_print:       total time =   57960.25 ms /   111 tokens


...

Llama.generate: 152 prefix-match hit, remaining 71 prompt tokens to eval
llama_perf_context_print:        load time =   89179.42 ms
llama_perf_context_print: prompt eval time =   30816.93 ms /    71 tokens (  434.04 ms per token,     2.30 tokens per second)
llama_perf_context_print:        eval time =   26801.12 ms /    38 runs   (  705.29 ms per token,     1.42 tokens per second)
llama_perf_context_print:       total time =   57640.44 ms /   109 tokens



🤖 AI Recommendation:

Product Name: Samsung Galaxy A06
Brand: Samsung
Color: N/A
Memory: N/A
Price: 31500

🔍 Enter your product query: Vivo Y33


Llama.generate: 152 prefix-match hit, remaining 67 prompt tokens to eval
llama_perf_context_print:        load time =   89179.42 ms
llama_perf_context_print: prompt eval time =   29290.19 ms /    67 tokens (  437.17 ms per token,     2.29 tokens per second)
llama_perf_context_print:        eval time =   26060.92 ms /    37 runs   (  704.35 ms per token,     1.42 tokens per second)
llama_perf_context_print:       total time =   55372.49 ms /   104 tokens



🤖 AI Recommendation:

Product Name: Vivo Y28
Brand: Vivo
Color: N/A
Memory: N/A
Price: 40499


KeyboardInterrupt: Interrupted by user

# **Satisfactory**

In [ ]:
import os
import re
import pandas as pd
import pickle
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from llama_cpp import Llama

# ----- Configuration -----
CSV_FILE = "product-data.csv"
EMBEDDING_FILE = "product_embeddings.pkl"
SIMILARITY_THRESHOLD = 0.55
PKR_RANGE_BUFFER = 5000
BRANDS = ['xiaomi', 'samsung', 'infinix', 'realme', 'oppo', 'vivo', 'tecno', 'nokia']

# Initialize models
embedder = SentenceTransformer("all-mpnet-base-v2")
llama = Llama(
    model_path="/content/llama-2-7b.Q4_K_M.gguf",
    n_ctx=2048,
    n_threads=4,
    chat_format="chatml",
    verbose=False
)

# ----- Data Loading -----
def load_product_data():
    if os.path.exists(EMBEDDING_FILE):
        with open(EMBEDDING_FILE, "rb") as f:
            df = pickle.load(f)
            required_cols = ['product_title', 'brand', 'price', 'features', 'embedding']
            if not all(col in df.columns for col in required_cols):
                raise ValueError("Corrupt embeddings file. Please delete and regenerate.")
            return df

    # Load and process CSV
    df = pd.read_csv(CSV_FILE)
    required_cols = ['product_title', 'brand', 'price', 'color', 'memory']
    if not all(col in df.columns for col in required_cols):
        missing = [col for col in required_cols if col not in df.columns]
        raise ValueError(f"Missing columns: {', '.join(missing)}")

    # Create features
    df["features"] = "Color: " + df["color"].str.title() + ", Memory: " + df["memory"].str.upper()
    df["brand"] = df["brand"].str.lower()

    # Generate embeddings
    print("🔨 Generating Pakistan-market embeddings...")
    descriptions = [
        f"{row['product_title']} | Brand: {row['brand']} | Price: PKR {row['price']} | {row['features']}"
        for _, row in df.iterrows()
    ]
    df["embedding"] = embedder.encode(descriptions, batch_size=32, show_progress_bar=True).tolist()

    # Save for future
    df[['product_title', 'brand', 'price', 'features', 'embedding']].to_pickle(EMBEDDING_FILE)
    return df
# ----- Price Extraction -----
def extract_price_range(query: str) -> tuple:
    """Improved price parser for Pakistani formats"""
    # Corrected regex pattern
    query = re.sub(r'[^\d.]', '', query.lower().replace('rs', '').replace('₹', '').replace(',', ''))

    # Find all price mentions
    numbers = [float(match) for match in re.findall(r'\d+\.?\d*', query)]

    # Handle conversion terms
    conversions = {
        'k': 1000,
        'thousand': 1000,
        'lakh': 100000,
        'hazar': 1000
    }
    for term, multiplier in conversions.items():
        if term in query:
            numbers = [n * multiplier for n in numbers]

    if not numbers:
        return (0, float('inf'))

    # Determine range logic
    if 'under' in query or 'below' in query:
        return (0, numbers[0] + PKR_RANGE_BUFFER)
    elif 'above' in query or 'over' in query:
        return (max(0, numbers[0] - PKR_RANGE_BUFFER), float('inf'))  # Fixed line
    elif len(numbers) >= 2:
        return (min(numbers), max(numbers) + PKR_RANGE_BUFFER)
    else:
        return (max(0, numbers[0] - PKR_RANGE_BUFFER), numbers[0] + PKR_RANGE_BUFFER)
# ----- Recommendation Engine -----
def recommend_product(query: str, df: pd.DataFrame) -> str:
    # Detect mentioned brands
    query_lower = query.lower()
    mentioned_brands = [brand for brand in BRANDS if brand in query_lower]

    # Filter by brand if mentioned
    if mentioned_brands:
        brand_filter = df['brand'].isin(mentioned_brands)
        df = df[brand_filter]
        if df.empty:
            return f"❌ No {mentioned_brands[0].title()} phones found. Try other brands."

    # Extract price range
    min_price, max_price = extract_price_range(query)
    filtered_df = df[(df['price'] >= min_price) & (df['price'] <= max_price)]

    if filtered_df.empty:
        return f"🤷 No phones in PKR {min_price:,}-{max_price:,}. Try wider range."

    # Calculate similarities
    query_emb = embedder.encode(query)
    filtered_df["similarity"] = cosine_similarity([query_emb], filtered_df["embedding"].tolist())[0]
    top_products = filtered_df.nlargest(3, "similarity")

    if top_products.iloc[0]["similarity"] < SIMILARITY_THRESHOLD:
        return ("🔍 Be specific:\n"
                f"- Price range (Current: PKR {min_price:,}-{max_price:,})\n"
                "- Brand preference\n"
                "- Features needed")

    # Prepare product info
    products_info = "\n\n".join([
        f"📦 {row['product_title']}\n"
        f"🏷️ Brand: {row['brand'].title()}\n"
        f"💵 Price: PKR {row['price']:,}\n"
        f"🚀 Features: {row['features']}"
        for _, row in top_products.iterrows()
    ])

    # LLM Prompt with brand context
    brand_context = f"focusing on {mentioned_brands[0].title()}" if mentioned_brands else "considering all brands"
    prompt = f"""<|im_start|>system
You're a smartphone expert {brand_context} in Pakistan. Rules:
1. Use ONLY these products:
{products_info}
2. Highlight features matching: {query}
3. Compare prices clearly in PKR
4. Use emoji bullet points (✔️, 🔥, 💡)
5. Keep under 150 words
6. Never mention unavailable products

Example Response:
"Top Xiaomi phones under PKR 50k:

📱 Redmi Note 12 Pro (PKR 49,999)
✔️ 120Hz AMOLED Display
✔️ 108MP OIS Camera
💡 Best camera in range

📱 Poco X5 Pro (PKR 47,499)
✔️ Snapdragon 778G
🔥 Top gaming performance

Need detailed specs?"<|im_end|>
<|im_start|>user
{query}<|im_end|>
<|im_start|>assistant
🇵🇰 {mentioned_brands[0].title() + ' Expert' if mentioned_brands else 'Phone Finder'}:
"""

    # Generate response
    response = llama(
        prompt,
        max_tokens=350,
        temperature=0.7,
        top_p=0.9,
        stop=["<|im_end|>"],
        echo=False
    )

    return response["choices"][0]["text"].strip()

# ----- User Interface -----
if __name__ == "__main__":
    try:
        df = load_product_data()
        print("\n🌟 Welcome to Pakistani Phone Finder! Ask like 'Best Xiaomi under 50k?'\n")

        while True:
            try:
                query = input("You: ").strip()
                if not query:
                    continue
                if query.lower() in ('exit', 'quit', 'bye'):
                    print("\nAssistant: Shukriya! Visit us for great deals! 🛍️")
                    break

                print("\nAssistant: ", end="", flush=True)
                response = recommend_product(query, df)
                print(response + "\n")

            except KeyboardInterrupt:
                print("\nAssistant: Session saved. Allah Hafiz! 👋")
                break

    except Exception as e:
        print(f"\n⚠️ Error: {str(e)}\nPlease check your data and try again.")

llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized



🌟 Welcome to Pakistani Phone Finder! Ask like 'Best Xiaomi under 50k?'

You: best phone under 60000rs

Assistant: 

<ipython-input-11-b2f2a0864d7f>:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["similarity"] = cosine_similarity([query_emb], filtered_df["embedding"].tolist())[0]


🔍 Be specific:
- Price range (Current: PKR 55,000.0-65,000.0)
- Brand preference
- Features needed


Assistant: Session saved. Allah Hafiz! 👋
